In [31]:
from lmfit import minimize, Parameters

import pandas as pd
import tellurium as te
import numpy as np

import matplotlib.pyplot as plt

import aesara.tensor as at
import aesara
floatX = aesara.config.floatX

import os 
os.chdir('..')
from emll.aesara_utils import LeastSquaresSolve
os.chdir('notebooks')

In [2]:
# import the data 
df = pd.read_csv('../data/Simplified_Teusink_yeast_1.05.csv')

In [3]:
r = te.loada('../models/Simplified_Teusink_yeast.ant')
r.steadyState()
N=r.getFullStoichiometryMatrix()

In [4]:
enzymes = ['e_' + i for i in r.getReactionIds()]
internal = r.getFloatingSpeciesIds()
external = r.getBoundarySpeciesIds()
fluxes = ['v_' + i for i in r.getReactionIds()]

v_star = df[fluxes].loc[0]

In [5]:
ex = r.getScaledElasticityMatrix()

a = r.getBoundarySpeciesIds()
b = r.getReactionIds()
c=[]
for i in b: 
    for ii in a: 
        c.append(r.getUnscaledParameterElasticity(i, ii) * r[ii]/r[i])
ey = np.array(c).reshape((len(b),len(a)))

In [19]:
en = df[enzymes].loc[1:len(r.getReactionIds())]/df[enzymes].loc[0]
xn = df[internal].loc[1:len(r.getReactionIds())]/df[internal].loc[0]
vn = df[fluxes].loc[1:len(r.getReactionIds())]/df[fluxes].loc[0]
yn = df[external].iloc[-(len(vn)):]/df[external].loc[0]


## Using lmfit to predict elasticity values w/o ey values

In [23]:
def cb(params, iter, resid, *args, **kws):
    print(iter, (resid**2).sum().sum())

def residual(params, xn, yn, v_star, df, *args, **kws):
    
    ex_params = [params[ii] for ii in [i for i in params]]
    ex = np.asarray(ex_params).reshape((16,11))
    
    a = (en.values*v_star.values)
    bb = np.ones((16,16))
    cc = ex@(np.log(xn)).T

    model = a.T*(bb + cc)

    return (df[fluxes].loc[1:len(r.getReactionIds())].values-model.T).values

    #model = np.diag(v_star)@(np.ones((16,19)) + ex@np.log(xn).T + ey@np.log(yn).T)
    #return (df[fluxes].loc[1:].values-model.T)

params = Parameters()

for i in range(16*(11)):
    params.add('Ex'+str(i), value=0, min=-10, max=10)

out = minimize(residual, params, args=(xn, yn, v_star, df))#, iter_cb=cb)

print(out.chisqr)
# out.params

7.162129346270159e-07


In [24]:
lmfit_ex = np.array([i.value for i in out.params.values()])
np.where(np.abs(lmfit_ex) > 0.01)


(array([  0,   1,  12,  13,  23,  34,  46,  47,  58,  59,  70,  71,  82,
         83,  94,  95, 106, 107, 118, 119, 130, 131, 142, 143, 164, 169],
       dtype=int64),)

In [25]:
out.params['Ex13'].value

-0.9540906588775258

In [26]:
ex

               GLCi,      G6P,       F6P,     F16P,      TRIO,       BPG,      P3G,       P2G,       PEP,       PYR,      ACE
vGLK   [[   1.44964, -1.43765,         0,        0,         0,         0,        0,         0,         0,         0,        0],
vPGI    [         0,  1.40347, -0.953057,        0,         0,         0,        0,         0,         0,         0,        0],
vGLYCO  [         0,        1,         0,        0,         0,         0,        0,         0,         0,         0,        0],
vTreha  [         0,        1,         0,        0,         0,         0,        0,         0,         0,         0,        0],
vPFK    [         0,        0,   1.38303, -1.03688,         0,         0,        0,         0,         0,         0,        0],
vALD    [         0,        0,         0,  1.10649, -0.823896,         0,        0,         0,         0,         0,        0],
vGAPDH  [         0,        0,         0,        0,  0.969712, -0.705532,        0,         0,         0, 

Now, we are going to explore control coefficient values for each set of elasticity values predicted by lmfit

In [40]:
def estimate_CCs(Ex):
    vn[vn == 0] = 1e-6
    
    a = np.diag(en.values / vn.values)
    a = np.diag(a)
    a = a[np.newaxis,:].repeat(1, axis=0)

    Ex_ss = a @ Ex
    As = N @ np.diag(v_star) @ Ex_ss
    bs = N @ np.diag(v_star)
    bs = bs[np.newaxis, :].repeat(1, axis=0)
    
    As = at.as_tensor_variable(As)
    bs = at.as_tensor_variable(bs)

    def solve_aesara(A, b):
        rsolve_op = LeastSquaresSolve()
        return rsolve_op(A, b).squeeze()

    CCC, _ = aesara.scan(lambda A, b: solve_aesara(A, b),
                        sequences=[As, bs], strict=True)

    identity = np.eye(len(N.T))
    identity = identity[np.newaxis,:].repeat(1, axis=0)
    
    FCC = (Ex_ss @ CCC.eval()) + identity
    
    return -FCC

In [41]:
lmfit_ex_df = pd.DataFrame(lmfit_ex.reshape((16,11)), index=r.getReactionIds(), columns=r.getFloatingSpeciesIds())
FCC_estimates = np.squeeze(estimate_CCs(lmfit_ex_df.values))

In [42]:
FCC_estimates[0]

array([-1.80606096e+00,  5.13373389e-02,  1.91238828e-02,  1.95488580e-01,
        3.53763380e-02,  3.31154584e-02,  1.61423796e-02,  1.22314548e-02,
        8.61649212e-03,  5.27647093e-03,  5.60849004e-03,  3.49773169e-03,
        3.15888202e-04,  4.07186661e-01,  7.61420852e-04,  1.19823764e-02])

In [39]:
r.getScaledFluxControlCoefficientMatrix()[0]

[1.97827702e-01 5.10757773e-02 1.89554933e-02 1.93767264e-01
 3.51968189e-02 3.29828174e-02 1.58676664e-02 1.20388199e-02
 8.49372163e-03 5.19193084e-03 5.57501480e-03 3.47188097e-03
 2.80782735e-04 4.06810162e-01 6.76802198e-04 1.17873451e-02]

In [43]:
FCC_estimates[5]

array([ 1.07719943e-01,  2.36109151e-01, -1.58929312e-02, -1.62461075e-01,
        1.62721844e-01, -1.84767916e+00,  7.42276875e-02,  5.62219789e-02,
        3.95763293e-02,  2.41947930e-02,  2.57223537e-02,  1.60017425e-02,
        1.40114919e-03,  2.23765138e-01,  3.37734744e-03,  5.49937106e-02])

In [44]:
r.getScaledFluxControlCoefficientMatrix()[5]

[ 0.10793611  0.23703552 -0.01581667 -0.1616815   0.1633435   0.15306863
  0.07363961  0.05587047  0.03941817  0.02409502  0.02587286  0.01611251
  0.00130307  0.22195833  0.00314094  0.05470341]

With the exception of the reaction being perturbed, the predicted FCC values match up quite well with the ground truth FCC values. 